In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd drive/My\ Drive/AML/aml-project

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
/content/drive/My Drive/AML/aml-project


In [0]:
! git pull

In [2]:
!pip install tensorflow==2.0 keras==2.3.1
!pip install spacy
!pip install nltk
!pip install gensim
import nltk
nltk.download('punkt')
nltk.download('stopwords')

     |████████████████████████████████| 86.3MB 53kB/s 
     |████████████████████████████████| 378kB 59.1MB/s 
     |████████████████████████████████| 3.8MB 65.6MB/s 
     |████████████████████████████████| 450kB 55.9MB/s 
     |████████████████████████████████| 81kB 11.1MB/s 
ERROR: tensorboard 2.0.2 has requirement grpcio>=1.24.3, but you'll have grpcio 1.15.0 which is incompatible.
ERROR: google-colab 1.0.0 has requirement google-auth~=1.4.0, but you'll have google-auth 1.10.1 which is incompatible.
  Found existing installation: google-auth 1.4.2
    Uninstalling google-auth-1.4.2:
      Successfully uninstalled google-auth-1.4.2
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
  Found existing installation: tensorflow 1.15.0
    Unin

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
%cd drive/My\ Drive/AML/aml-project

[Errno 2] No such file or directory: 'drive/My Drive/AML/aml-project'
/content/drive/My Drive/AML/aml-project


In [4]:
import os
import re
import json
import numpy as np
import spacy
import gensim
import argparse
import itertools
import multiprocessing as mp
from gensim.models import Word2Vec, FastText
from pandas import pandas as pd

from utils import preprocess
from utils import parse_content_line

import logging  # Setting up the loggings to monitor gensim

logging.basicConfig(
    format="%(levelname)s - %(asctime)s: %(message)s",
    datefmt="%H:%M:%S",
    level=logging.INFO,
)

attrs = ["title"]
attributes = [attr + "_left" for attr in attrs] + [attr + "_right" for attr in attrs]
dataset_paths = ["./dataset/computers/train/computers_train_xlarge.json", "./dataset/computers/test/computers_gs.json"]
dataset = []
print("* LOADING DATASET")
for dataset_path in dataset_paths:
  dataset.extend(np.concatenate(
      [
          parse_content_line(x, attributes=attributes, label=0)
          for x in open(dataset_path, "r").readlines()
      ],
      axis=0,
  ).astype(object))
print("* DONE")
sentences = list(itertools.chain(*dataset))

Using TensorFlow backend.


* LOADING DATASET
* DONE


In [5]:
cores = mp.cpu_count()  # Count the number of cores in a computer
print("* PREPROCESS")
preprocess_method = 'nltk'
# Preprocess text
if preprocess_method == "spacy":
    # Load spacy for tokenizing text
    nlp = spacy.load("en_core_web_sm", disable=["ner", "parser"])
    txt = [
        preprocess(doc, method=preprocess_method)
        for doc in nlp.pipe(sentences, batch_size=5000, n_threads=cores)
    ]
elif preprocess_method == "nltk":
    with mp.Pool(processes=cores) as pool:
        for attr in range(len(attributes)):
            txt = pool.map(preprocess, sentences)
print("* DONE")

# Remove duplicates
cleaned_sentences = pd.DataFrame({"sentences": txt})
cleaned_sentences = cleaned_sentences.dropna().drop_duplicates()

# Prepare sentences for w2v/fasttext training
sentences = [row.split() for row in cleaned_sentences["sentences"]]

* PREPROCESS
* DONE


In [6]:
sentences[:5]

[['hp', 'intel', 'xeon', 'x5560', 'prijzen', 'tweakers'],
 ['495906', 'b21', 'hp', 'x5560', '2', '80ghz', 'ml350', 'g6'],
 ['495906',
  'b21',
  'hp',
  'x5560',
  '2',
  '80ghz',
  'ml350',
  'g6',
  'new',
  'wholesale',
  'price'],
 ['asus',
  'motherboard',
  'lga2066',
  'ddr4',
  '2',
  'u',
  'atx',
  '2xgbe',
  'prime',
  'x299',
  'deluxe'],
 ['asus', 'prime', 'x299', 'deluxe']]

In [11]:
# Train W2V or FastText
print("* TRAIN EMBEDDINGS")
size = 100
min_count = 1
context_window = 9
epochs = 300
algorithm = "w2v"
if algorithm == "w2v":
    model = Word2Vec(
        sg=1,  #  Use SKIPGRAM model
        hs=0,  # Don't use hierarchical softmax
        min_count=min_count,  # All words that have an absolute frequency < 20 will be discarded
        window=context_window,  # Context-window size
        size=size,  # Embeddings dimension
        sample=0,
        alpha=0.03,
        min_alpha=0.0007,
        negative=10,  # How many negative samples will be sampled for each positive example
        workers=cores,
        compute_loss=True,
    )
elif algorithm == "fasttext":
    model = FastText(
        sg=1,  #  Use SKIPGRAM model
        hs=0,  # Don't use hierarchical softmax
        min_count=min_count,  # All words that have an absolute frequency < 20 will be discarded
        window=context_window,  # Context-window size
        size=size,  # Embeddings dimension
        sample=0,
        alpha=0.03,
        min_alpha=0.0007,
        negative=10,  # How many negative samples will be sampled for each positive example
        workers=cores,
        word_ngrams=1,
        min_n=3,
        max_n=7,
    )

model.build_vocab(sentences, progress_per=10000)
model.train(sentences, total_examples=model.corpus_count, epochs=epochs, report_delay=1)
print("* DONE")

INFO - 12:04:04: collecting all words and their counts
INFO - 12:04:04: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO - 12:04:04: collected 4764 word types from a corpus of 53824 raw words and 4293 sentences
INFO - 12:04:04: Loading a fresh vocabulary
INFO - 12:04:04: effective_min_count=1 retains 4764 unique words (100% of original 4764, drops 0)
INFO - 12:04:04: effective_min_count=1 leaves 53824 word corpus (100% of original 53824, drops 0)
INFO - 12:04:04: deleting the raw counts dictionary of 4764 items
INFO - 12:04:04: sample=0 downsamples 0 most-common words
INFO - 12:04:05: downsampling leaves estimated 53824 word corpus (100.0% of prior 53824)
INFO - 12:04:05: estimated required memory for 4764 words and 100 dimensions: 6193200 bytes
INFO - 12:04:05: resetting layer weights


* TRAIN EMBEDDINGS


INFO - 12:04:05: training model with 2 workers on 4764 vocabulary and 100 features, using sg=1 hs=0 sample=0 negative=10 window=9
INFO - 12:04:06: worker thread finished; awaiting finish of 1 more threads
INFO - 12:04:06: worker thread finished; awaiting finish of 0 more threads
INFO - 12:04:06: EPOCH - 1 : training on 53824 raw words (53824 effective words) took 0.5s, 108584 effective words/s
INFO - 12:04:06: worker thread finished; awaiting finish of 1 more threads
INFO - 12:04:06: worker thread finished; awaiting finish of 0 more threads
INFO - 12:04:06: EPOCH - 2 : training on 53824 raw words (53824 effective words) took 0.5s, 108950 effective words/s
INFO - 12:04:07: worker thread finished; awaiting finish of 1 more threads
INFO - 12:04:07: worker thread finished; awaiting finish of 0 more threads
INFO - 12:04:07: EPOCH - 3 : training on 53824 raw words (53824 effective words) took 0.5s, 106220 effective words/s
INFO - 12:04:07: worker thread finished; awaiting finish of 1 more th

* DONE


In [12]:
if algorithm == "w2v":
    sentences_dict = {}
    with open(
        "./dataset/embeddings/w2v/w2v_"
        + "_".join(attrs)
        + "_"
        + str(epochs)
        + "Epochs_"
        + str(context_window)
        + "ContextWindow_"
        + str(min_count)
        + "MinCount_"
        + str(size)
        + "d.txt",
        "w",
    ) as f:
        for sentence in sentences:
            for token in sentence:
                try:
                    sentences_dict[token]
                except KeyError:
                    sentences_dict[token] = 1
                    if token in model.wv.vocab:
                        f.write(
                            token
                            + " "
                            + " ".join(str(x) for x in model.wv.get_vector(token))
                        )
                        f.write("\n")
        f.flush()
        f.close()
    model.wv.save_word2vec_format(
        "./dataset/embeddings/w2v/w2v_"
        + "_".join(attrs)
        + "_"
        + str(epochs)
        + "Epochs_"
        + str(context_window)
        + "ContextWindow_"
        + str(min_count)
        + "MinCount_"
        + str(size)
        + "d.bin",
        binary=True,
    )
elif algorithm == "fasttext":
    sentences_dict = {}
    with open(
        "./dataset/embeddings/fasttext/fasttext_"
        + "_".join(attrs)
        + "_"
        + str(epochs)
        + "Epochs_"
        + str(context_window)
        + "ContextWindow_"
        + str(min_count)
        + "MinCount_"
        + str(size)
        + "d.txt",
        "w",
    ) as f:
        for sentence in sentences:
            for token in sentence:
                try:
                    sentences_dict[token]
                except KeyError:
                    sentences_dict[token] = 1
                    try:
                        f.write(
                            token
                            + " "
                            + " ".join(str(x) for x in model.wv.get_vector(token))
                        )
                        f.write("\n")
                    except KeyError:
                        continue
        f.flush()
        f.close()
    model.wv.save(
        "./dataset/embeddings/fasttext/fasttext_"
        + "_".join(attrs)
        + "_"
        + str(epochs)
        + "Epochs_"
        + str(context_window)
        + "ContextWindow_"
        + str(min_count)
        + "MinCount_"
        + str(size)
        + "d.bin"
    )

INFO - 12:06:34: storing 4764x100 projection weights into ./dataset/embeddings/w2v/w2v_title_300Epochs_9ContextWindow_1MinCount_100d.bin
/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
